In [2]:
%load_ext autoreload
%autoreload 2
import numpy as np
import xarray as xr
from matplotlib import pyplot as plt
import pandas as pd
import sys
sys.path.append('/home/onno/Thesis/Scripts')
import my_tools
from my_tools import plot_dic, file_dic
%matplotlib qt

In [3]:
#set path
path = '/media/onno/Algemeen/Thesis/GFS_T850/'

In [4]:
#loading GFS and ERA5 data for specific box 
file_gfs = 'gefsrf2_fldmean_t850_control0-252h_24hourly_lon_6_12_lat_52_46.nc'
file_era = 'era51_fldmean_mars_t850_79-19_24hourly_lon_6_12_lat_52_46.nc'
file_clim_p90 = 'era51_mars_t850_79-19_24hourly_90p_lon_6_12_lat_52_46_SMOOTHED.nc'
file_clim_p10 = 'era51_mars_t850_79-19_24hourly_10p_lon_6_12_lat_52_46_SMOOTHED.nc'
gfs = xr.open_dataset(path+file_gfs,decode_times=False).squeeze()
#set gfs time data to datetime object
init_time_gfs = pd.Timestamp('1800-01-01')
gfs['time'] = [pd.Timedelta(i,'hours')+init_time_gfs for i in gfs.time.values]
gfs['lead'] = gfs['lead']//24
#group forecast by lead time and take daily means
gfs = gfs.sel(lead=slice(0,9)).groupby('lead').mean()
era = xr.open_dataset(path+file_era).squeeze()
#load percentile climatology. For convenience I set the data to all days in the leap year 2016. That has no further effect on the data
clim_p90 = xr.open_dataset(path+file_clim_p90).squeeze()
clim_p90['time']=pd.date_range('2016-01-01',"2016-12-31")
clim_p10 = xr.open_dataset(path+file_clim_p10).squeeze()
clim_p10['time']=pd.date_range('2016-01-01',"2016-12-31")


General monthly Gilbert Skill Scores

In [75]:
#put climatologies in list to loop over them
climz = [clim_p90,clim_p10]
#set and select lead day. Value must range between 0 and 9
lead_dayz = [1,3,5,7,9]
#make reanalysis and forecast of equal length
df_warm = pd.DataFrame(index=np.arange(1,13),columns=['day_1','day_3','day_5','day_7','day_9'])
df_cold = pd.DataFrame(index=np.arange(1,13),columns=['day_1','day_3','day_5','day_7','day_9'])
#loop over heat and cold extremes

for lead_day in lead_dayz:
    gfs_lead = gfs.sel(lead=lead_day)
    #convert time of forecast to initial time + forecast lead time
    gfs_lead['time'] = gfs.time + pd.Timedelta(lead_day,'days')
    era_lead = era.sel(time=slice(pd.Timestamp('1984-12-{:02d}'.format(lead_day+1)),
                             pd.Timestamp('2019-11-30')))
    gfs_lead = gfs_lead.sel(time=slice(pd.Timestamp('1984-12-{:02d}'.format(lead_day+1)),
                             pd.Timestamp('2019-11-30')))
    for j,clim in enumerate(climz):
        column = columnz[j]
        #calculate anomalies from climatoogy: see my_tools.py for more documentation
        era_anom = my_tools.calculate_anomalies(era_lead,clim)    
        gfs_anom = my_tools.calculate_anomalies(gfs_lead,clim)

        if j ==0:
            #convert anomalies to boolean array. If temperature is extreme than value is converted to True
            era_bool = era_anom.t>0
            gfs_bool = gfs_anom.t>0
            #fill in contingency table and group all values by month
            hits = (era_bool & gfs_bool).astype(np.int64).groupby('time.month').sum()
            miss = ((~gfs_bool)&(era_bool)).astype(np.int64).groupby('time.month').sum()
            false_alarm = ((gfs_bool)&(~era_bool)).astype(np.int64).groupby('time.month').sum()
            non_event = ((~gfs_bool)&(~era_bool)).astype(np.int64).groupby('time.month').sum()
            #caluclate Gilbert Skill Score. See my_tools.py for more documentation
            GSS = my_tools.gilbert_skill_score(hits,miss,false_alarm,non_event)
            #Write result to csv file
            df_warm['day_{}'.format(lead_day)]=pd.Series(GSS,index=np.arange(1,13))
        else:
            #convert anomalies to boolean array. If temperature is extreme than value is converted to True
            era_bool = era_anom.t<0
            gfs_bool = gfs_anom.t<0
            #fill in contingency table and group all values by month
            hits = (era_bool & gfs_bool).astype(np.int64).groupby('time.month').sum()
            miss = ((~gfs_bool)&(era_bool)).astype(np.int64).groupby('time.month').sum()
            false_alarm = ((gfs_bool)&(~era_bool)).astype(np.int64).groupby('time.month').sum()
            non_event = ((~gfs_bool)&(~era_bool)).astype(np.int64).groupby('time.month').sum()
            #caluclate Gilbert Skill Score. See my_tools.py for more documentation
            GSS = my_tools.gilbert_skill_score(hits,miss,false_alarm,non_event)
            #Write result to csv file
            df_cold['day_{}'.format(lead_day)]=pd.Series(GSS,index=np.arange(1,13))
df_warm.to_csv(path+'GSS/GSS_general_warm.csv')
df_cold.to_csv(path+'GSS/GSS_general_cold.csv')

Specific heat waves

In [5]:
#load all heat extremes
file_pers_hw = 'persistent_heatwaves_lon_6_12_lat_52_46.npy'
file_pers_cw = 'persistent_coldwaves_lon_6_12_lat_52_46.npy'
file_short_hw = 'short_heatwaves_lon_6_12_lat_52_46.npy'
file_short_cw = 'short_coldwaves_lon_6_12_lat_52_46.npy'
pers_hw = np.load(path+file_pers_hw)
pers_cw = np.load(path+file_pers_cw)
short_hw = np.load(path+file_short_hw)
short_cw = np.load(path+file_short_cw)
temp_extremez = [pers_hw,short_hw,pers_cw,short_cw]

Caluclate GSS for every single event and save overview

In [47]:
columnz = ['persistent_hw','short_hw','persistent_cw','short_cw',]
index = pd.date_range(pd.Timestamp('1984-12-01'),pd.Timestamp('2019-11-30'))
df = pd.DataFrame(index=index,columns=columnz,dtype=float)
lead_dayz = [1,3,5,7,9]
for lead_day in lead_dayz:
    for j,temp_extreme in enumerate(temp_extremez):
        column = columnz[j]
        for i,date in enumerate(temp_extreme[:,0]):
            if date<pd.Timestamp('1984-12-01')+pd.Timedelta(lead_day,'days'):
                continue
            begin_date = date - pd.Timedelta(lead_day,'days')
            end_date = date + pd.Timedelta(9-lead_day,'days')
            era_sub = era.sel(time=slice(begin_date,end_date)).load()
            gfs_sub = gfs.sel(time=begin_date).load()
            timez = [pd.Timestamp('2016-{:02d}-{:02d}'.format(i.month,i.day)) for i in pd.date_range(begin_date,end_date)]
            if j<2:
                clim_p90_sub = clim_p90.sel(time=timez).load()
                era_bool = era_sub.t>clim_p90_sub.t.values
                gfs_bool = gfs_sub.t>clim_p90_sub.t.values
            else:
                clim_p10_sub = clim_p10.sel(time=timez).load()
                era_bool = era_sub.t<clim_p10_sub.t.values
                gfs_bool = gfs_sub.t<clim_p10_sub.t.values
            hits = (era_bool & gfs_bool.values).astype(np.int64).sum()
            miss = ((~gfs_bool)&(era_bool.values)).astype(np.int64).sum()
            false_alarm = ((gfs_bool)&(~era_bool.values)).astype(np.int64).sum()
            non_event = ((~gfs_bool)&(~era_bool.values)).astype(np.int64).sum()            
            GSS = my_tools.gilbert_skill_score(hits,miss,false_alarm,non_event)
            df.loc[date][column]=GSS

    df = df.dropna(how='all')
    df.to_csv(path+'GSS/Gilbert_Skill_Score_lead_day_{}.csv'.format(lead_day))
#     df_month = df.groupby(df.index.month,dropna=True).mean()
#     df_month.to_csv(path+'GSS/Gilbert_Skill_Score_Monthly_Mean_lead_day_{}.csv'.format(lead_day))
df_count = df.groupby(df.index.month,dropna=True).count()
df_count.to_csv(path+'GSS/Temp_extremes_count.csv')

    

Calculate GSS based on monthly grouped contingency table

In [7]:
columnz = ['persistent_hw','short_hw','persistent_cw','short_cw',]
index = pd.date_range(pd.Timestamp('1984-12-01'),pd.Timestamp('2019-11-30'))
df = pd.DataFrame(index=np.arange(1,13),columns=columnz,dtype=float)
lead_dayz = [1,3,5,7,9]
for lead_day in lead_dayz:
    for j,temp_extreme in enumerate(temp_extremez):
        column = columnz[j]
        df_contingency = pd.DataFrame(columns=['hits','miss','false_alarm','non_event'])
        for i,date in enumerate(temp_extreme[:,0]):
            if date<pd.Timestamp('1984-12-01')+pd.Timedelta(lead_day,'days'):
                continue
            begin_date = date - pd.Timedelta(lead_day,'days')
            end_date = date + pd.Timedelta(9-lead_day,'days')
            era_sub = era.sel(time=slice(begin_date,end_date)).load()
            gfs_sub = gfs.sel(time=begin_date).load()
            timez = [pd.Timestamp('2016-{:02d}-{:02d}'.format(i.month,i.day)) for i in pd.date_range(begin_date,end_date)]
            if j<2:
                clim_p90_sub = clim_p90.sel(time=timez).load()
                era_bool = era_sub.t>clim_p90_sub.t.values
                gfs_bool = gfs_sub.t>clim_p90_sub.t.values
            else:
                clim_p10_sub = clim_p10.sel(time=timez).load()
                era_bool = era_sub.t<clim_p10_sub.t.values
                gfs_bool = gfs_sub.t<clim_p10_sub.t.values
            hits = (era_bool & gfs_bool.values).astype(np.int64).sum()
            miss = ((~gfs_bool)&(era_bool.values)).astype(np.int64).sum()
            false_alarm = ((gfs_bool)&(~era_bool.values)).astype(np.int64).sum()
            non_event = ((~gfs_bool)&(~era_bool.values)).astype(np.int64).sum()    
            df_contingency.loc[date] = [int(hits),int(miss),int(false_alarm),int(non_event)]

        df_contingency_month = df_contingency.groupby(df_contingency.index.month).sum()
        GSS = my_tools.gilbert_skill_score_dataframe(df_contingency_month)
        df[column]=GSS
    df.to_csv(path+'GSS/GSS_Temp_Extremes_Monthly_Grouped_lead_day_{}.csv'.format(lead_day))

    

SystemExit: 

/home/onno/miniconda3/envs/thesis/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3426: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


Calculate GSS based on seasonally grouped contingency table

In [21]:
columnz = ['persistent_hw','short_hw','persistent_cw','short_cw',]
index = pd.date_range(pd.Timestamp('1984-12-01'),pd.Timestamp('2019-11-30'))
df = pd.DataFrame(index=['DJF','JJA','MAM','SON'],columns=columnz,dtype=float)
month_to_season_lu = np.array([
    None,
    'DJF', 'DJF',
    'MAM', 'MAM', 'MAM',
    'JJA', 'JJA', 'JJA',
    'SON', 'SON', 'SON',
    'DJF'
])
lead_dayz = [1,3,5,7,9]
for lead_day in lead_dayz:
    print(' Lead Day is {}'.format(lead_day))
    for j,temp_extreme in enumerate(temp_extremez):
        column = columnz[j]
        df_contingency = pd.DataFrame(columns=['hits','miss','false_alarm','non_event'])
        for i,date in enumerate(temp_extreme[:,0]):
            if date<pd.Timestamp('1984-12-01')+pd.Timedelta(lead_day,'days'):
                continue
            begin_date = date - pd.Timedelta(lead_day,'days')
            end_date = date + pd.Timedelta(9-lead_day,'days')
            era_sub = era.sel(time=slice(begin_date,end_date)).load()
            gfs_sub = gfs.sel(time=begin_date).load()
            timez = [pd.Timestamp('2016-{:02d}-{:02d}'.format(i.month,i.day)) for i in pd.date_range(begin_date,end_date)]
            if j<2:
                clim_p90_sub = clim_p90.sel(time=timez).load()
                era_bool = era_sub.t>clim_p90_sub.t.values
                gfs_bool = gfs_sub.t>clim_p90_sub.t.values
            else:
                clim_p10_sub = clim_p10.sel(time=timez).load()
                era_bool = era_sub.t<clim_p10_sub.t.values
                gfs_bool = gfs_sub.t<clim_p10_sub.t.values
            hits = (era_bool & gfs_bool.values).astype(np.int64).sum()
            miss = ((~gfs_bool)&(era_bool.values)).astype(np.int64).sum()
            false_alarm = ((gfs_bool)&(~era_bool.values)).astype(np.int64).sum()
            non_event = ((~gfs_bool)&(~era_bool.values)).astype(np.int64).sum()    
            df_contingency.loc[date] = [int(hits),int(miss),int(false_alarm),int(non_event)]
        if (lead_day==9)&(j==0):
            sys.exit()

        df_contingency_month = df_contingency.groupby(month_to_season_lu[df_contingency.index.month]).sum()
        GSS = my_tools.gilbert_skill_score_dataframe(df_contingency_month)
        df[column]=GSS
    print(df)
#     df.to_csv(path+'GSS/GSS_Temp_Extremes_Seasonally_Grouped_lead_day_{}.csv'.format(lead_day))

    

 Lead Day is 1
     persistent_hw  short_hw  persistent_cw  short_cw
DJF       0.502900  0.449966       0.483407  0.495750
JJA       0.418036  0.480549       0.473479  0.558146
MAM       0.370017  0.436537       0.452485  0.460602
SON       0.483156  0.474639       0.509434  0.504272
 Lead Day is 3
     persistent_hw  short_hw  persistent_cw  short_cw
DJF       0.376064  0.425636       0.448351  0.318102
JJA       0.467139  0.347509       0.419476  0.373443
MAM       0.255781  0.336178       0.430113  0.387298
SON       0.423699  0.329568       0.308949  0.436907
 Lead Day is 5
     persistent_hw  short_hw  persistent_cw  short_cw
DJF       0.239121  0.349593       0.347947  0.247583
JJA       0.382122  0.302763       0.213424  0.235503
MAM       0.228985  0.262387       0.298077  0.245261
SON       0.261954  0.271464       0.288873  0.328387
 Lead Day is 7
     persistent_hw  short_hw  persistent_cw  short_cw
DJF       0.278046  0.268758       0.319754  0.201078
JJA       0.119264  0.

SystemExit: 

/home/onno/miniconda3/envs/thesis/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3426: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [50]:
lead_dayz = [1,3,5,7,9]
file = 'GSS/Gilbert_Skill_Score_lead_day_{}.csv'
month_to_season_lu = np.array([
    None,
    'DJF', 'DJF',
    'MAM', 'MAM', 'MAM',
    'JJA', 'JJA', 'JJA',
    'SON', 'SON', 'SON',
    'DJF'
])
for lead_day in lead_dayz:
    df = pd.read_csv(path+file.format(lead_day),index_col=0)
    df.index = pd.to_datetime(df.index)
    df_seasonally = df.groupby(month_to_season_lu[df.index.month]).mean()
    sys.exit()
    

SystemExit: 

/home/onno/miniconda3/envs/thesis/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3426: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
